In [ ]:
from triton_joseph_proj import *
import numpy as np
import matplotlib.pyplot as plt
import time
import array_api_compat.torch as xp

dict_data = np.load("/home/user/triton/slices_8_lors_40.npy", allow_pickle=True).item()

xstart = xp.asarray(dict_data['xstart']).cuda()
xend = xp.asarray(dict_data['xend']).cuda()
img_origin = xp.asarray(dict_data['img_origin']).cuda()
voxel_size = xp.asarray(dict_data['voxel_size']).cuda()
img_shape = dict_data['img_shape']
x_gt = xp.asarray(dict_data['x']).cuda()
meas = xp.asarray(dict_data['meas']).cuda() + 100

fwd = lambda x: joseph3d_fwd_vec(xstart.reshape(-1,3), xend.reshape(-1,3), x, img_origin, voxel_size, img_shape).reshape(meas.shape)

back = lambda y: joseph3d_back_vec(xstart.reshape(-1,3), xend.reshape(-1,3), y.ravel(), img_origin, voxel_size, img_shape)

tmp = fwd(x_gt)
torch.cuda.synchronize(x_gt.device)
start = time.time()
for i in range(10):
    tmp = fwd(x_gt)
    torch.cuda.synchronize(x_gt.device)
end_t = time.time()

sens_img = back(xp.ones_like(meas))
x = xp.ones_like(sens_img)
torch.cuda.synchronize(x_gt.device)

start = time.time()
for i in range(10):
    tmp = back(meas)
    torch.cuda.synchronize(x_gt.device)
end_t = time.time()

start = time.time()
for i in range(10000):
    em_precond = x/sens_img
    ratio = meas/(fwd(x)+100.)
    bp_ratio = back(ratio)
    x = em_precond*bp_ratio
end_t = time.time()

xp.save(x.cpu(), "triton_joseph_proj_recon.torch")


In [2]:
import torch
x_parallel_proj = torch.load("parallel_proj_recon.torch")
x = torch.load("triton_joseph_proj_recon.torch")
import matplotlib.pyplot as plt
from ipywidgets import interact
def show_slices(slice_idx):
    plt.figure(figsize=(18, 6), dpi=80)
    plt.subplot(1,3,1)
    plt.imshow(x[:,slice_idx,:].cpu(), cmap='gray')
    plt.colorbar()
    plt.subplot(1,3,2)
    plt.imshow(x_parallel_proj[:,slice_idx,:].cpu(), cmap='gray')
    plt.colorbar()
    plt.subplot(1,3,3)
    plt.imshow((x[:,slice_idx,:].cpu()-x_parallel_proj[:,slice_idx,:].cpu()), cmap='gray')
    plt.colorbar()
    plt.show()
    
interact(show_slices, slice_idx=(0, x.shape[1]-1, 1))

interactive(children=(IntSlider(value=3, description='slice_idx', max=7), Output()), _dom_classes=('widget-int…

<function __main__.show_slices(slice_idx)>

In [ ]:
x_parallel_proj = torch.load("parallel_proj_recon.torch")
x = torch.load("triton_joseph_proj_recon.torch")
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 3), dpi=80)
slice_idx = x.shape[1]//2
plt.subplot(1,3,1)
plt.imshow(x[:,slice_idx,:].cpu(), cmap='gray')
plt.title("Triton Joseph Proj") 
plt.axis('off')
plt.colorbar()
plt.subplot(1,3,2)
plt.imshow(x_parallel_proj[:,slice_idx,:].cpu(), cmap='gray')
plt.title("Parallel Proj")
plt.axis('off')
plt.colorbar()
plt.subplot(1,3,3)
plt.imshow((x[:,slice_idx,:].cpu()-x_parallel_proj[:,slice_idx,:].cpu()).abs(), cmap='gray')
plt.title("Absolute Difference")
plt.axis('off')
plt.colorbar()
# make tight
plt.tight_layout()
plt.savefig("validation_10000_MLEM.png", dpi=300)
plt.show()